In [2]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
from shapely.geometry import Point
from shapely.geometry import MultiPoint
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon
from shapely.geometry import box
from descartes.patch import PolygonPatch
import ipywidgets as widgets
import ipywidgets.widgets.interaction
#from ipywidgets import interactive, fixed
from ipywidgets import *
import random
import pandas as pd

import re

from shapely import geometry
import pickle
import time

from ipyparallel import Client

from shapely import affinity
from shapely.geometry import LineString
import warnings
warnings.simplefilter('ignore', np.RankWarning)
import gc
gc.enable()


In [3]:
#os.chdir('/home/esb/Projects/uJ/local/experiments/HT-CAZ/')
#os.chdir('/home/charly/Lab/Projects/uJ/local/experiments/HT-CAZ/')



In [4]:
pwd()

'/home/esb/Projects/uJ/local/experiments/HT-CAZ'

In [5]:
#sys.path.insert(0, '/Users/ESB/RPM_SYNC/RPM_Work/uJ_master/current/uJ_src/python/')
#sys.path.insert(0, '/home/esb/Projects/uJ/uJ_master/current/uJ_src/python/')
#sys.path.insert(0, '/Users/ESB/SYNC_RPM/RPM_Work/uJ_master/current/uJ_src/python/')
#sys.path.insert(0, '/home/charly/Lab/Projects/uJ/uJ_master/current/uJ_src/python/')
sys.path.insert(0, '/home/charly/Lab/Projects/uJ/local/current/uJ_src_pHT/python/')
sys.path.insert(0, '/home/esb/Projects/uJ/local/current/uJ_src_pHT/python/')




In [6]:
from readroi import *
from DataManagers import *
from TrackingFunctions import *
from PlottingFunctions import *

Importing readroi
DataStructs...loaded!
DataManagers...loaded!
TrackingFunctions... loaded!
PlottingFunctions... loaded!


In [7]:
# import sys, importlib
# importlib.reload(sys.modules['DataManagers'])
# importlib.reload(sys.modules['TrackingFunctions'])
# from DataManagers import *
# from TrackingFunctions import *

## Note 
### Start a cluster with ipcluster start

## Experiment Parameters

In [21]:
expeLabel="HT-CAZ"
#list_pos="xy07,xy08,xy09,xy12,xy13,xy17,xy26,xy27,xy32,xy34".split(",")  
#list_pos="xy07,xy08,xy09,xy12,xy13,xy14,xy15,xy17,xy24,xy25,xy26,xy27,xy28,xy29,xy30,xy31,xy32,xy33,xy34,xy35,xy36".split(",")



expeLabel="HT-CAZ"
rootDir="../../data_sample/HT-CAZ/"
trap_list="xy07,xy08,xy09,xy12,xy13,xy14,xy15,xy17,xy24,xy25,xy26,xy27,xy28,xy29,xy30,xy31,xy32,xy33,xy34,xy35,xy36".split(",")  
trap_list="xy34,xy34-1".split(",")  
#trap="xy34" 


figxlim=640
figylim=512

flims=(figxlim,figylim)

data_type="normData"


#Input files
for trap in trap_list:
    #Input files
    dirNameROIs=rootDir+"data_rois/"+trap+"/DsRed+GFP/"
    #dirNameDATA=rootDir+'data/'+data_type+'/'+trap+"/"
    dirNameDATA=rootDir+'data/'+data_type+'/'+trap+"/"
    
    #Output
    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
    if not os.path.exists(dirNameCELLS):
        print("making dir cells...")
        os.makedirs(dirNameCELLS)

    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    if not os.path.exists(dirNameTRACKEDCELLS):
        print("making dir tracked cells...")
        os.makedirs(dirNameTRACKEDCELLS)



    dirNameTRACKEDCELLS_IMGS=rootDir+'images_cells_tracked/'+trap+'/'+data_type+'/'
    if not os.path.exists(dirNameTRACKEDCELLS_IMGS):
        print("making dir tracked cells imgs ...")
        os.makedirs(dirNameTRACKEDCELLS_IMGS)

    dirNameDATALINEAGES=rootDir+'data/'
    if not os.path.exists(dirNameDATALINEAGES):
        print("making dir lineages...")
        os.makedirs(dirNameDATALINEAGES)
        
    dirNameFIGURES=rootDir+'figures/traked_cells/'+trap+'/'+data_type+'/'
    if not os.path.exists(dirNameFIGURES):
        print("making dir figures...")
        os.makedirs(dirNameFIGURES)

#toFile=False
toFile=True


frame2min=10
frame_experiment_start=37;
frame_signal_start=int(91-(frame_experiment_start-1)/2)
frame_signal_end=int(127-(frame_experiment_start-1)/2)
#frame_experiment_end=int(145-(frame_experiment_start-1)/2)
#frame_experiment_end=int(163-(frame_experiment_start-1)/2)
frame_experiment_end=163

#frame_experiment_end=145
frame_experiment_start=37;
frame_signal_start=int(91-(frame_experiment_start-1)/2)
frame_signal_end=int(127-(frame_experiment_start-1)/2)
frame_experiment_end=int(145-(frame_experiment_start-1)/2)
frame_experiment_end=163

frame_experiment_start=55;
frame_signal_start=91
frame_signal_end=127
frame_experiment_end=127


t_experiment_start=0
t_signal_start=(frame_signal_start-frame_experiment_start)*frame2min  
t_signal_end=(frame_signal_end-frame_experiment_start)*frame2min
t_experiment_end=(frame_experiment_end-frame_experiment_start)*frame2min


frames=range(int(frame_experiment_start),int(frame_experiment_end+1))

times=[(this_frame-frame_experiment_start)*frame2min for this_frame in frames]
times_hour=[(this_frame-frame_experiment_start)*frame2min/60 for this_frame in frames]
#print("\nMinutes:  %s"%times)
#print("\nHours:  [%s,%s]"%(times_hour[0],times_hour[-1]))


print("\nFrames:  %s\t %s->%s hours"%(frames,times_hour[0],times_hour[-1]))
print("t_experiment_start=%s\t(%sh)"%(t_experiment_start, t_experiment_start/60))
print("t_experiment_end=%s\t(%sh)"%(t_experiment_end, t_experiment_end/60))


#Prepare file system
dataPath="%sdata/DsRed+GFP/"%rootDir

#if not os.path.exists(figurePathDM) and toFile:
#    os.mkdir(figurePathDM)

frames=range(int(frame_experiment_start),int(frame_experiment_end)+1)
frames_list=[x for x in frames]


Frames:  range(55, 128)	 0.0->12.0 hours
t_experiment_start=0	(0.0h)
t_experiment_end=720	(12.0h)


___
# Create Cells


### Load ROIs generated by uJ

In [9]:

fileROIs=load_rois(dirNameROIs)
dataFiles=load_data_files(dirNameDATA)
numFrames=len(fileROIs)
print('Loading %s frames'%numFrames)
print('Loading %s data files'%len(dataFiles))

fileROIs=fileROIs[0:frame_experiment_end]
dataFiles=dataFiles[0:frame_experiment_end]
#fileROIs=fileROIs[0:5]
#dataFiles=dataFiles[0:5]

numFrames=len(fileROIs)
print('Loading  %s frames'%numFrames)
print('Loading %s data files'%len(dataFiles))

    
   ##### NOTE if path not found check the NAME of your local sync folder 

FileNotFoundError: [Errno 2] No such file or directory: '../../data_sample/HT-CAZ/data_rois/xy34-1/DsRed+GFP/'

### Process ROIs (parallel version)

In [10]:
def par_process_rois():
    
    import numpy as np
    
    def get_max_x_angle(poly):
        rangle=0
        angles= list(range(0,185,5))
        (bminx, bminy, bmaxx, bmaxy)=poly.bounds
        xl=bmaxx-bminx
        yl=bmaxy-bminy
        center=Point((xl/2)+bminx,(yl/2)+bminy)

        max_xrange=xl
        for i,angle in enumerate(angles):
            #print(i,angle)
            poly_t=affinity.rotate(poly,angle,center)
            (bminx, bminy, bmaxx, bmaxy)=poly_t.bounds
            xl=bmaxx-bminx

            if(xl>max_xrange):
                max_xrange=xl
                rangle=angle

        return rangle,center


    def get_cell_axis(poly):
      #  t=time.time()
        line=LineString()
        pre_cloud=[]
        #poly=poly.convex_hull
#         if(poly.geom_type=="Point"):
            
#             poly=poly.buffer(1)
        angle,center=get_max_x_angle(poly)

        poly=affinity.rotate(poly, angle,center )

        ext=poly.exterior.coords

        decs=0

        for i,seed_point in enumerate(ext):
            pre_cloud_i=[]

            seed_point=Point(seed_point)

            i=.5;
            intersection=poly.exterior.intersection(seed_point)
            sphere=seed_point.buffer(i)
            intersection=poly.exterior.intersection(sphere.exterior)
            while(len(list(intersection))>0):
                arc=Point()
                sphere=seed_point.buffer(i)
                intersection=poly.exterior.intersection(sphere.exterior)
                i+=3
                #i+=.5

                if(len(list(intersection))<=1):
                    continue

                arc=poly.intersection(sphere.exterior)

                if(arc.geom_type=="MultiLineString"):

                    for this_geom in arc.geoms:
                        arc_xy=list(this_geom.coords)

                        for pi in arc_xy:
                            pi=Point(pi)
                            pre_cloud_i.append((round(pi.x,decs),round(pi.y,decs)))

                elif(arc.geom_type=="LineString"):
                    for pi in list(arc.coords):
                        pi=Point(pi) 
                        pre_cloud_i.append((round(pi.x,decs),round(pi.y,decs)))


            pre_cloud.append(pre_cloud_i)


        cloud=MultiPoint(pre_cloud[0])

        for line in pre_cloud:

            line=MultiPoint(line)
            cloud=cloud.union(line)

        ###make poly fit to points cloud
        xs=[]
        ys=[]
        for pi in cloud:
            pi=Point(pi)

            xs.append(pi.x)
            ys.append(pi.y)

        pf=np.poly1d(np.polyfit(xs,ys,25))
        xr=np.linspace(np.min(xs),np.max(xs),100)

        line=[]
        for x in xr:
            pi=Point(x,pf(x))
            line.append(pi)

        line=LineString(line)
        linexy=np.array(line)
        linex=linexy[:,0]
        liney=linexy[:,1]

        line=poly.intersection(line)
        liner=affinity.rotate(line,-angle,center)
        if(liner.geom_type=="MultiLineString"):
            line=LineString()
            for this_geom in liner.geoms:
                if(this_geom.length>line.length):
                    line=this_geom
            liner=line



        linexy=np.array(liner)
        linex=linexy[:,0]
        liney=linexy[:,1]

        x=(np.max(xs)-np.min(xs))/2+np.min(xs)
        pi=Point(x,pf(x))


        center=affinity.rotate(pi,-angle,center)
       # t=time.time()-t
       # print(t)

        return liner,center
    
    frame_axis=[]
    frame_centers=[]
    for j,this_roi in enumerate(rois):
        print(j,this_roi)
        this_axis,this_center=get_cell_axis(this_roi)
            
        frame_axis.append(this_axis)
        frame_centers.append(this_center)
    ret=[rois, roiIDs, frame_axis, frame_centers]
    return ret



In [12]:
re_do_cells=False
re_do_cells=True

trap_cells=[]

#trap_list="xy03".split(",")  #temp no 06

trap_list="xy34".split(",")  

for trap in trap_list:
    dirNameROIs=rootDir+"data_rois/"+trap+"/DsRed+GFP/"
    dirNameDATA=rootDir+'data/'+data_type+'/'+trap+"/"
    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
    fileROIs=load_rois(dirNameROIs)
    dataFiles=load_data_files(dirNameDATA)
    
    fx=frame_experiment_start-37
    fileROIs=fileROIs[fx:]
    #dataFiles=dataFiles[fx:]
    
    numFrames=len(fileROIs)
    numData=len(dataFiles)
    
    
    print('Loading %s frames'%numFrames)
    print('Loading %s data files'%numData)
    #print('Loading %s data files'%len(dataFiles))

#    fileROIs=fileROIs[0:frame_experiment_end-frame_experiment_start+1]
 #   dataFiles=dataFiles[0:frame_experiment_end-frame_experiment_start+1]
    
    
  #  fileROIs=fileROIs[19:20]
 #   dataFiles=dataFiles[19:20]
    
#     fx=frame_experiment_end-frame_experiment_start
#     fileROIs=fileROIs[fx:frame_experiment_end-frame_experiment_start+1]
#     dataFiles=dataFiles[fx:frame_experiment_end-frame_experiment_start+1]



    #fileROIs=fileROIs[0:2]
    #dataFiles=dataFiles[0:2]

    #numFrames=len(fileROIs)
    #print('Loading  %s frames'%numFrames)
    #print('Loading %s data files'%len(dataFiles))



    if re_do_cells:
        rc = Client()
        dview = rc[:]
        with rc[:].sync_imports(): 
            from shapely.geometry import Point
            from shapely.geometry import LineString
            from shapely import affinity
            from shapely.geometry import MultiPoint


        global cells  # global?
        cells=[]
        for frame, this_file in enumerate(fileROIs):
            [rois,roiIDs]=load_frame('%s%s'%(dirNameROIs,this_file))
            this_data_file="%s%s"%(dirNameDATA,dataFiles[frame])
            
            #rois=[poly.convex_hull for poly in rois]
            
            print("\n(Parallel) Processing %s ROIs from %s"%(len(rois),this_file))


            tic()

            #First scatter rois
            dview.scatter('rois', rois)
            dview.scatter('roiIDs', roiIDs)

            #Asynchronous execution of par_process_rois
            ret=dview.apply_async(par_process_rois)

            #Now gather results
            frame_rois = []
            frame_roiIDs = []
            frame_axis = []
            frame_center = []

            ret_proc=ret.get()
            for i in rc.ids:
                frame_rois.append(ret_proc[i][0])
                frame_roiIDs.append(ret_proc[i][1])
                frame_axis.append(ret_proc[i][2])
                frame_center.append(ret_proc[i][3])

            frame_rois=[y for x in frame_rois for y in x]
            frame_roiIDs=[y for x in frame_roiIDs for y in x]
            frame_axis=[y for x in frame_axis for y in x]
            frame_center=[y for x in frame_center for y in x]

            toc()

            print("Compiling fluorescence from %s"%(dataFiles[frame]))
            this_cells=create_cells(frame,this_data_file, frame_roiIDs, frame_rois, frame_axis, frame_center,frame_experiment_start)
            cells.append(this_cells)

            fileNameCells="%s%s.pkl"%(dirNameCELLS,os.path.splitext(os.path.basename(this_file))[0])    
            save_cells(this_cells, fileNameCells)
    
        trap_cells.append(cells)
    
    else:
        print()
        #max_frames=-1
        cells=load_cells(dirNameCELLS)
        trap_cells.append(cells)
        ##### NOTE   if not working check for changes in your folder names due to sync

Loading 73 frames
Loading 73 data files
importing Point from shapely.geometry on engine(s)
importing LineString from shapely.geometry on engine(s)
importing affinity from shapely on engine(s)
importing MultiPoint from shapely.geometry on engine(s)

(Parallel) Processing 562 ROIs from HT-CAZ_xy34_055.zip


/home/esb/.anaconda3/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@strix')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


Elapsed time: 46.406061 seconds.
Compiling fluorescence from HT-CAZ_xy34_055.txt
Saving 562 cells to ../../data_sample/HT-CAZ/data_cells/xy34/normData/HT-CAZ_xy34_055.pkl

(Parallel) Processing 557 ROIs from HT-CAZ_xy34_056.zip
Elapsed time: 44.164365 seconds.
Compiling fluorescence from HT-CAZ_xy34_056.txt
Saving 557 cells to ../../data_sample/HT-CAZ/data_cells/xy34/normData/HT-CAZ_xy34_056.pkl

(Parallel) Processing 553 ROIs from HT-CAZ_xy34_057.zip
Elapsed time: 43.650619 seconds.
Compiling fluorescence from HT-CAZ_xy34_057.txt
Saving 553 cells to ../../data_sample/HT-CAZ/data_cells/xy34/normData/HT-CAZ_xy34_057.pkl

(Parallel) Processing 551 ROIs from HT-CAZ_xy34_058.zip
Elapsed time: 43.932256 seconds.
Compiling fluorescence from HT-CAZ_xy34_058.txt
Saving 551 cells to ../../data_sample/HT-CAZ/data_cells/xy34/normData/HT-CAZ_xy34_058.pkl

(Parallel) Processing 559 ROIs from HT-CAZ_xy34_059.zip
Elapsed time: 46.501267 seconds.
Compiling fluorescence from HT-CAZ_xy34_059.txt
Saving 

Elapsed time: 47.626770 seconds.
Compiling fluorescence from HT-CAZ_xy34_091.txt
Saving 541 cells to ../../data_sample/HT-CAZ/data_cells/xy34/normData/HT-CAZ_xy34_091.pkl

(Parallel) Processing 551 ROIs from HT-CAZ_xy34_092.zip
Elapsed time: 45.989329 seconds.
Compiling fluorescence from HT-CAZ_xy34_092.txt
Saving 551 cells to ../../data_sample/HT-CAZ/data_cells/xy34/normData/HT-CAZ_xy34_092.pkl

(Parallel) Processing 542 ROIs from HT-CAZ_xy34_093.zip
Elapsed time: 41.847256 seconds.
Compiling fluorescence from HT-CAZ_xy34_093.txt
Saving 542 cells to ../../data_sample/HT-CAZ/data_cells/xy34/normData/HT-CAZ_xy34_093.pkl

(Parallel) Processing 531 ROIs from HT-CAZ_xy34_094.zip
Elapsed time: 38.860063 seconds.
Compiling fluorescence from HT-CAZ_xy34_094.txt
Saving 531 cells to ../../data_sample/HT-CAZ/data_cells/xy34/normData/HT-CAZ_xy34_094.pkl

(Parallel) Processing 543 ROIs from HT-CAZ_xy34_095.zip
Elapsed time: 41.020224 seconds.
Compiling fluorescence from HT-CAZ_xy34_095.txt
Saving 

Elapsed time: 74.787592 seconds.
Compiling fluorescence from HT-CAZ_xy34_127.txt
Saving 317 cells to ../../data_sample/HT-CAZ/data_cells/xy34/normData/HT-CAZ_xy34_127.pkl


## View cells

In [ ]:
%matplotlib inline 

cells_plots_t=[]
cells=trap_cells[0]
for tracked_frame in range(len(cells)):
    this_fig=draw_cell(cells,tracked_frame+1)
    cells_plots_t.append(this_fig)



In [ ]:
%matplotlib inline 

this_tracked_plots=cells_plots_t
interactive_plot_t=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=1,max=len(this_tracked_plots),step=1,value=1))
interactive_plot_t    



___
# uJ_Tracker: Step 2

### Select cells to track

In [ ]:
%matplotlib ipympl
#%matplotlib widgets

 #if plot does not show,uncomment this line run, comment and re-run
def onclickSelectMulti(event):

       #global max_frames

       #First find if Point is inside a roi
    this_point = Point(event.xdata,event.ydata)
    #if(event.y)
    
    title=event.inaxes.get_title()
    tindex=trap_list.index(title)
    to_track_polys=trap_to_track_polys[tindex]
    ax=axs[tindex]
    for select_index, select_poly in enumerate(to_track_polys):
        in_roi=select_poly.contains(this_point)
        if in_roi:
            select_x,select_y = select_poly.exterior.xy     
            ax.plot(select_x, select_y, color='#6699cc', alpha=0.7, linewidth=2, solid_capstyle='round', zorder=3)
            ax.axis('off')
            trap_toTrack_roiIndex[tindex].append(select_index)
            trap_toTrack_roiFrame[tindex].append(to_track_frame)
            #trap_toTrack_trapIndex
            
            break



fig,axs=plt.subplots(len(trap_cells),1,figsize=(30,30))
trap_to_track_polys=[]

trap_toTrack_roiIndex=[]
trap_toTrack_roiFrame=[]

for ti,t_cells in enumerate(trap_cells):
    cells=t_cells
    to_track_frame=len(cells)-1
    #to_track_frame=14
   # global toTrack_roiIndex
    trap_toTrack_roiIndex.append([])
    trap_toTrack_roiFrame.append([])




    
    #fig=plt.figure()
    ax = axs[ti]

    ax.set_xlim(0,640)
    ax.set_ylim(0,512)
    #ax.clear()
    to_track_polys=[]

    for this_cell in cells[to_track_frame]:
        this_poly=this_cell['roiPoly']
        to_track_polys.append(this_poly)
        patch = PolygonPatch(this_poly, facecolor=[0.75,0.75,0.75], edgecolor=[0,0,0], alpha=0.7, zorder=2)
        ax.add_patch(patch)
       #ax.plot(x=100,y=100,c='r*')
    #ax.axis('off')
    trap_to_track_polys.append(to_track_polys)
    ax.set_title(trap_list[ti])
    ax.set_aspect('equal')
    #fig.canvas.layout.width = '1280px'
    fig.canvas.layout.height = '%spx'%(512*len(trap_cells))
    fig.canvas.layout.width = '640px'
    #fig.canvas.layout.height = '512px'
    
    


cid = fig.canvas.mpl_connect('button_press_event', onclickSelectMulti)
#cid = plt.gcf().canvas.mpl_connect('button_press_event', onclickSelect)
    
   

### Track selected cells

In [ ]:
print(trap_toTrack_roiIndex)
print(trap_toTrack_roiFrame)

In [ ]:
#print(toTrack_roiIndex)
#print(toTrack_roiFrame)
weight_vec=[-5,-100,200,1500,-5,-5]
#weight_vec=[-1,-5,5,35,-175]
#[distance,angle_diff,coverage,axisFraction,GFPDiff,DsRedDiff]
trap_tracked_cells=[]
for ti,t_cells in enumerate(trap_cells):
    cells=t_cells
    cells=clear_tracks(cells)
    
    cells_tracked=cells.copy()
    if('cells_tracked' in vars()):
        cells_tracked=clear_tracks(cells_tracked)
    toTrack_roiIndex=trap_toTrack_roiIndex[ti]
    toTrack_roiFrame=trap_toTrack_roiFrame[ti]
    
    cells_tracked=track_this_indexs(toTrack_roiIndex,toTrack_roiFrame,cells,weight_vec) 
    
    trap_tracked_cells.append(cells_tracked)
    print('\n\nTracking complete! Trap:',trap)
    
    print("\nSaving cells...")
    file_cells= list(f for f in os.listdir(dirNameCELLS) if f.endswith('.pkl'))
    file_cells.sort()
    for frame, this_file in enumerate(file_cells):
        this_cells=cells_tracked[frame] 
        #print(frame, this_file)
        fileNameTrackedCells="%s%s.pkl"%(dirNameTRACKEDCELLS,os.path.splitext(os.path.basename(this_file))[0])    
    #    print(fileNameFrameCells)
        save_cells(this_cells, fileNameTrackedCells)




## Track cells from last frame

In [ ]:
#trap_list2=[trap_list[0]]
trap_tracked_cells=[]
for tindex,trap in enumerate(trap_list):

    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    
    
    cells=trap_cells[tindex]
    
    cells=clear_tracks(cells)
    cells_tracked=cells.copy()
    

    
    max_frames=len(cells)-1
    #max_frames=len(cells)-1

    from_frame=max_frames
    to_frame=max_frames-1
    
    for this_frame in range(from_frame,to_frame,-1):
        toTrack_roiIndex=[]
        toTrack_roiFrame=[]
        for select_index, this_cell in enumerate(cells_tracked[this_frame]):
            if len(this_cell['trackID'])==0:
                toTrack_roiFrame.append(this_frame)
                toTrack_roiIndex.append(select_index)
        print('\n___\nTracking %s cells from frame %s'%(len(toTrack_roiFrame),this_frame))
        cells_tracked=track_this_indexs(toTrack_roiIndex,toTrack_roiFrame,cells_tracked,weight_vec) 
    print('\n\nTracking complete! Trap:',trap)
    trap_tracked_cells.append(cells_tracked)
    print("\nSaving cells...")
    file_cells= list(f for f in os.listdir(dirNameCELLS) if f.endswith('.pkl'))
    file_cells.sort()
    for frame, this_file in enumerate(file_cells):
        this_cells=cells_tracked[frame] 
        #print(frame, this_file)
        fileNameTrackedCells="%s%s.pkl"%(dirNameTRACKEDCELLS,os.path.splitext(os.path.basename(this_file))[0])    
    #    print(fileNameFrameCells)
        save_cells(this_cells, fileNameTrackedCells)

### Track all cells all frames

In [14]:
#weight_vec=[-5,-100,200,1500,-5,-5]

#weight_vec=[-5,-1,150,150,-5,-5]
weight_vec=[-5,-100,200,1500,-5,-5]

trap_tracked_cells=[]

for tindex,trap in enumerate(trap_list):
    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    
    
    cells=trap_cells[tindex]
    
    cells=clear_tracks(cells)
    cells_tracked=cells.copy()
    
    max_frames=len(cells)-1
    from_frame=max_frames
    to_frame=0
    
    for this_frame in range(from_frame,to_frame,-1):
        toTrack_roiIndex=[]
        toTrack_roiFrame=[]
        for select_index, this_cell in enumerate(cells_tracked[this_frame]):
            if len(this_cell['trackID'])==0:
                toTrack_roiFrame.append(this_frame)
                toTrack_roiIndex.append(select_index)
        print('\n___\nTracking %s cells from frame %s'%(len(toTrack_roiFrame),this_frame))
        cells_tracked=track_this_indexs(toTrack_roiIndex,toTrack_roiFrame,cells_tracked,weight_vec) 
    print('\n\nTracking complete! Trap:',trap)
    trap_tracked_cells.append(cells_tracked)
    print("\nSaving cells...")
    file_cells= list(f for f in os.listdir(dirNameCELLS) if f.endswith('.pkl'))
    file_cells.sort()
    for frame, this_file in enumerate(file_cells):
        this_cells=cells_tracked[frame] 
        #print(frame, this_file)
        fileNameTrackedCells="%s%s.pkl"%(dirNameTRACKEDCELLS,os.path.splitext(os.path.basename(this_file))[0])    
    #    print(fileNameFrameCells)
        save_cells(this_cells, fileNameTrackedCells)
    


___
Tracking 317 cells from frame 72

0....................................................................................................
100....................................................................................................
200....................................................................................................
300.................
___
Tracking 48 cells from frame 71

0................................................
___
Tracking 57 cells from frame 70

0.........................................................
___
Tracking 57 cells from frame 69

0.........................................................
___
Tracking 57 cells from frame 68

0.........................................................
___
Tracking 52 cells from frame 67

0....................................................
___
Tracking 72 cells from frame 66

0........................................................................
___
Tracking 78 cells from frame 65

0...............

0.............................................................................................
___
Tracking 79 cells from frame 4

0...............................................................................
___
Tracking 70 cells from frame 3

0......................................................................
___
Tracking 70 cells from frame 2

0......................................................................
___
Tracking 88 cells from frame 1

0........................................................................................

Tracking complete! Trap: xy34

Saving cells...
Saving 562 cells to ../../data_sample/HT-CAZ/data_cells_tracked/xy34/normData/HT-CAZ_xy34_055.pkl
Saving 557 cells to ../../data_sample/HT-CAZ/data_cells_tracked/xy34/normData/HT-CAZ_xy34_056.pkl
Saving 553 cells to ../../data_sample/HT-CAZ/data_cells_tracked/xy34/normData/HT-CAZ_xy34_057.pkl
Saving 551 cells to ../../data_sample/HT-CAZ/data_cells_tracked/xy34/normData/HT-CAZ_xy34_058.pkl
Saving

In [15]:
trap_list2=trap_list.copy()
trap_list2=["xy34-1"]
trap_cells2=trap_cells.copy()

#trap_tracked_cells=[]
for tindex,trap in enumerate(trap_list2):

    dirNameCELLS=rootDir+'data_cells/'+trap+'/'+data_type+'/'
    dirNameCELLS=rootDir+'data_cells/xy34/'+data_type+'/'
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    print("Tracking trap: ",trap)
    
    cells=trap_cells2[tindex]
    
    cells=clear_tracks(cells)
    cells_tracked=cells.copy()
    

    
    max_frames=len(cells)-1
    #max_frames=len(cells)-1

    from_frame=max_frames
    to_frame=max_frames-1
    
    for this_frame in range(from_frame,to_frame,-1):
        toTrack_roiIndex=[]
        toTrack_roiFrame=[]
        for select_index, this_cell in enumerate(cells_tracked[this_frame]):
            if len(this_cell['trackID'])==0:
                toTrack_roiFrame.append(this_frame)
                toTrack_roiIndex.append(select_index)
        print('\n___\nTracking %s cells from frame %s'%(len(toTrack_roiFrame),this_frame))
        cells_tracked=track_this_indexs(toTrack_roiIndex,toTrack_roiFrame,cells_tracked,weight_vec) 
    print('\n\nTracking complete! Trap:',trap)
    trap_tracked_cells.append(cells_tracked)
    print("\nSaving cells...")
    file_cells= list(f for f in os.listdir(dirNameCELLS) if f.endswith('.pkl'))
    file_cells.sort()
    for frame, this_file in enumerate(file_cells):
        this_cells=cells_tracked[frame] 
        #print(frame, this_file)
        fileNameTrackedCells="%s%s.pkl"%(dirNameTRACKEDCELLS,os.path.splitext(os.path.basename(this_file))[0])    
    #    print(fileNameFrameCells)
        save_cells(this_cells, fileNameTrackedCells)


Tracking trap:  xy34-1

___
Tracking 317 cells from frame 72

0....................................................................................................
100....................................................................................................
200....................................................................................................
300.................

Tracking complete! Trap: xy34-1

Saving cells...
Saving 562 cells to ../../data_sample/HT-CAZ/data_cells_tracked/xy34-1/normData/HT-CAZ_xy34_055.pkl
Saving 557 cells to ../../data_sample/HT-CAZ/data_cells_tracked/xy34-1/normData/HT-CAZ_xy34_056.pkl
Saving 553 cells to ../../data_sample/HT-CAZ/data_cells_tracked/xy34-1/normData/HT-CAZ_xy34_057.pkl
Saving 551 cells to ../../data_sample/HT-CAZ/data_cells_tracked/xy34-1/normData/HT-CAZ_xy34_058.pkl
Saving 559 cells to ../../data_sample/HT-CAZ/data_cells_tracked/xy34-1/normData/HT-CAZ_xy34_059.pkl
Saving 578 cells to ../../data_sample/HT-CAZ/data_cells_t

### Load tracked cells

In [22]:
trap_list="xy34,xy34-1".split(",")  

In [23]:
trap_tracked_cells=[]
for trap in trap_list:
    
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+'/'+data_type+'/'
    print(dirNameTRACKEDCELLS)
    tracked_cells=load_cells(dirNameTRACKEDCELLS)
    trap_tracked_cells.append(tracked_cells)

../../data_sample/HT-CAZ/data_cells_tracked/xy34/normData/
72: Loading HT-CAZ_xy34_127.pkl
../../data_sample/HT-CAZ/data_cells_tracked/xy34-1/normData/
72: Loading HT-CAZ_xy34_127.pkl


### Draw ROIs + Tracked Cells

In [24]:
%matplotlib inline
#max_frames=len(cells_tracked)
trap_tracked_plots=[]
for tindex,trap in enumerate(trap_list):
    tracked_plots=[]
    cells_tracked=trap_tracked_cells[tindex]
    dirNameFIGURES=rootDir+'figures/traked_cells/'+trap+'/'+data_type+'/'
    print("Trap: %s in %s"%(trap,dirNameFIGURES))
    for i,tracked_frame in enumerate(frames_list):
        this_fig=draw_cell(cells_tracked,i+1,flims)
        #plt.show(this_fig)
        tracked_frame_name=str(tracked_frame).zfill(3)
        figname="%s%s_%s-%s.png"%(dirNameFIGURES,expeLabel,trap,tracked_frame_name)
        this_fig.savefig(figname)
        tracked_plots.append(this_fig)
       # display(this_fig)
        plt.close("all")
    print()
    trap_tracked_plots.append(tracked_plots)

Trap: xy34 in ../../data_sample/HT-CAZ/figures/traked_cells/xy34/normData/
Building plot for frame frame 73 
Trap: xy34-1 in ../../data_sample/HT-CAZ/figures/traked_cells/xy34-1/normData/
Building plot for frame frame 73 


In [25]:

this_tracked_plots=trap_tracked_plots[0]
#interactive_plot=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=1,max=max_frames,step=1,value=1),fs=fixed(frame_experiment_start))
interactive_plot=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=frame_experiment_start,max=frame_experiment_end,step=1,value=1),fs=fixed(frame_experiment_start))
interactive_plot    


interactive(children=(IntSlider(value=55, description='tracked_frame', max=127, min=55), Output()), _dom_class…

In [26]:

this_tracked_plots=trap_tracked_plots[1]
#interactive_plot=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=1,max=max_frames,step=1,value=1),fs=fixed(frame_experiment_start))
interactive_plot=interactive(show_all,this_tracked_plots=fixed(this_tracked_plots),tracked_frame=widgets.IntSlider(min=frame_experiment_start,max=frame_experiment_end,step=1,value=1),fs=fixed(frame_experiment_start))
interactive_plot    


interactive(children=(IntSlider(value=55, description='tracked_frame', max=127, min=55), Output()), _dom_class…

In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
 display: none;
}
</style>